In [1]:
#*----------------------------------------------------------------------------*
#* Copyright (C) 2024 ETH Zurich, Switzerland                                 *
#* SPDX-License-Identifier: Apache-2.0                                        *
#*                                                                            *
#* Licensed under the Apache License, Version 2.0 (the "License");            *
#* you may not use this file except in compliance with the License.           *
#* You may obtain a copy of the License at                                    *
#*                                                                            *
#* http://www.apache.org/licenses/LICENSE-2.0                                 *
#*                                                                            *
#* Unless required by applicable law or agreed to in writing, software        *
#* distributed under the License is distributed on an "AS IS" BASIS,          *
#* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.   *
#* See the License for the specific language governing permissions and        *
#* limitations under the License.                                             *
#*                                                                            *
#* Author:  Thorir Mar Ingolfsson                                             *
#*----------------------------------------------------------------------------*
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

features = np.load('../data/x_train_250.npy')
labels = np.load('../data/y_train_binary_250.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)

# Average CV score on the training set was: 0.903792748162954
exported_pipeline = ExtraTreesClassifier(bootstrap=True, criterion="gini", max_features=0.15000000000000002, 
                                         min_samples_leaf=5, min_samples_split=3, n_estimators=100, n_jobs=-1)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250 Hz Binary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

250 Hz Binary:
Accuracy score: 0.9100758396533044
F1 Score: 0.8358763981937845


In [2]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
features = np.load('../data/x_train_250_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_250_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8798373679340494
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeClassifier(criterion="gini", max_depth=7, min_samples_leaf=9, 
                                                       min_samples_split=11)),
    DecisionTreeClassifier(criterion="gini", max_depth=10, min_samples_leaf=11, min_samples_split=13)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250 Hz MultiBinary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

/home/thoriri/miniconda3/envs/TPOT/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


250 Hz MultiBinary:
Accuracy score: 0.8798426450450609
F1 Score: 0.4496729473786976


In [3]:
from sklearn.naive_bayes import MultinomialNB
features = np.load('../data/x_train_250_unrolled.npy')
labels = np.load('../data/y_train_multioutput_250_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8718424598961854
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=MultinomialNB(alpha=1.0, fit_prior=False)),
    DecisionTreeClassifier(criterion="gini", max_depth=10, min_samples_leaf=13, min_samples_split=18)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250 Hz MultiOutput:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))

250 Hz MultiOutput:
Accuracy score: 0.8716970564303623
F1 Score: 0.8373526630506835


In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.preprocessing import MinMaxScaler
features = np.load('../data/x_train_250_1000.npy')
labels = np.load('../data/y_train_binary_250_1000.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8928981773960551
exported_pipeline = make_pipeline(
    SelectPercentile(score_func=f_classif, percentile=95),
    MinMaxScaler(),
    GradientBoostingClassifier(learning_rate=0.5, max_depth=7, max_features=0.05, min_samples_leaf=6, min_samples_split=4, n_estimators=100, subsample=0.8500000000000001)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250Hz + 1000Hz Binary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

250Hz + 1000Hz Binary:
Accuracy score: 0.8897879733599305
F1 Score: 0.7975412985017288


In [5]:
from sklearn.linear_model import LogisticRegression


features = np.load('../data/x_train_250_1000_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_250_1000_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.879984057844875
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LogisticRegression(C=0.01, dual=False, penalty="l2")),
    DecisionTreeClassifier(criterion="gini", max_depth=10, min_samples_leaf=4, min_samples_split=9)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250Hz + 1000Hz MultiBinary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

250Hz + 1000Hz MultiBinary:
Accuracy score: 0.880335399293927
F1 Score: 0.45401740882718955


In [6]:
from sklearn.preprocessing import StandardScaler
features = np.load('../data/x_train_250_1000_unrolled.npy')
labels = np.load('../data/y_train_multioutput_250_1000_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8718812037324135
exported_pipeline = make_pipeline(
    StandardScaler(),
    DecisionTreeClassifier(criterion="gini", max_depth=10, min_samples_leaf=9, min_samples_split=2)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)



exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250Hz + 1000Hz MultiOutput:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))

250Hz + 1000Hz MultiOutput:
Accuracy score: 0.8720930062505301
F1 Score: 0.8382886376735099


In [7]:
features = np.load('../data/x_train_256.npy')
labels = np.load('../data/y_train_binary_256.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)

# Average CV score on the training set was: 0.8345366502734926
exported_pipeline = ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.5, min_samples_leaf=5, min_samples_split=2, n_estimators=100, n_jobs=-1)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz Binary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

256Hz Binary:
Accuracy score: 0.8432237585409157
F1 Score: 0.8019303969548669


In [8]:
from sklearn.neighbors import KNeighborsClassifier
features = np.load('../data/x_train_256_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_256_unrolled.npy')
# Average CV score on the training set was: 0.8525547915406534
exported_pipeline = KNeighborsClassifier(n_neighbors=41, p=1, weights="distance")
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz MultiBinary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

256Hz MultiBinary:
Accuracy score: 0.8217033410448162
F1 Score: 0.7691557536918362


In [9]:
from sklearn.preprocessing import StandardScaler
features = np.load('../data/x_train_256_unrolled.npy')
labels = np.load('../data/y_train_multioutput_256_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8438450363093619
exported_pipeline = KNeighborsClassifier(n_neighbors=25, p=1, weights="uniform")
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)



exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz MultiOutput:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))

256Hz MultiOutput:
Accuracy score: 0.8441163667666062
F1 Score: 0.7947964886031196


In [10]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
features = np.load('../data/x_train_256_512.npy')
labels = np.load('../data/y_train_binary_256_512.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)

# Average CV score on the training set was: 0.8380404987178351
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.2, min_samples_leaf=3, min_samples_split=3, n_estimators=100)),
    StackingEstimator(estimator=SGDClassifier(alpha=0.001, eta0=1.0, fit_intercept=False, l1_ratio=0.5, learning_rate="constant", loss="squared_hinge", penalty="elasticnet", power_t=0.1)),
    DecisionTreeClassifier(criterion="gini", max_depth=6, min_samples_leaf=9, min_samples_split=17)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz + 512Hz Binary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))



features = np.load('../data/x_train_256_512_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_256_512_unrolled.npy')
# Average CV score on the training set was: 0.8435785268442461
exported_pipeline = KNeighborsClassifier(n_neighbors=42, p=1, weights="distance")
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)
    
exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz + 512Hz MultiBinary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))



features = np.load('../data/x_train_256_512_unrolled.npy')
labels = np.load('../data/y_train_multioutput_256_512_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8332207928593143
exported_pipeline = make_pipeline(
    VarianceThreshold(threshold=0.01),
    KNeighborsClassifier(n_neighbors=39, p=1, weights="uniform")
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz + 512Hz MultiOutput:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))

256Hz + 512Hz Binary:
Accuracy score: 0.8413099484740388
F1 Score: 0.8076853797658361
256Hz + 512Hz MultiBinary:
Accuracy score: 0.8185691636940151
F1 Score: 0.7725183252577961
256Hz + 512Hz MultiOutput:
Accuracy score: 0.8334421843470741
F1 Score: 0.7793764645849337


In [11]:
from sklearn.kernel_approximation import Nystroem
from tpot.builtins import ZeroCount
features = np.load('../data/x_train_all.npy')
labels = np.load('../data/y_train_binary_all.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)

# Average CV score on the training set was: 0.8642657669243821
exported_pipeline = make_pipeline(
    Nystroem(gamma=0.7000000000000001, kernel="linear", n_components=7),
    ExtraTreesClassifier(bootstrap=True, criterion="gini", max_features=0.9000000000000001, min_samples_leaf=4, min_samples_split=4, n_estimators=100, n_jobs=-1)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("All frequencies Binary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))



features = np.load('../data/x_train_all_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_all_unrolled.npy')
# Average CV score on the training set was: 0.8725976631067676
exported_pipeline = DecisionTreeClassifier(criterion="gini", max_depth=10, min_samples_leaf=20, min_samples_split=9)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)
    
exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("All frequencies MultiBinary:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))



features = np.load('../data/x_train_all_unrolled.npy')
labels = np.load('../data/y_train_multioutput_all_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8608947377109661
exported_pipeline = make_pipeline(
    ZeroCount(),
    DecisionTreeClassifier(criterion="gini", max_depth=10, min_samples_leaf=8, min_samples_split=5)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("All frequencies MultiOutput:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))


All frequencies Binary:
Accuracy score: 0.8683960047711419
F1 Score: 0.7862301113568312
All frequencies MultiBinary:
Accuracy score: 0.8543355944047518
F1 Score: 0.7615760826694012
All frequencies MultiOutput:
Accuracy score: 0.8608604836982803
F1 Score: 0.8235654043842016


In [12]:
from sklearn.naive_bayes import BernoulliNB
features = np.load('../data/x_train_250_temporal.npy')
labels = np.load('../data/y_train_binary_250_temporal.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)

# Average CV score on the training set was: 0.9353538974885545
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=BernoulliNB(alpha=100.0, fit_prior=False)),
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.5, min_samples_leaf=2, min_samples_split=12, n_estimators=100)),
    ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.8, min_samples_leaf=5, min_samples_split=5, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250 Hz Binary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

features = np.load('../data/x_train_250_temporal_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_250_temporal_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.9001021630924988
exported_pipeline = ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.35000000000000003, min_samples_leaf=1, min_samples_split=4, n_estimators=100)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250 Hz MultiBinary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

features = np.load('../data/x_train_250_temporal_unrolled.npy')
labels = np.load('../data/y_train_multioutput_250_temporal_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8916159701788221
exported_pipeline = ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.8, min_samples_leaf=1, min_samples_split=11, n_estimators=100)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250 Hz MultiOutput (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))

250 Hz Binary (Temporal):
Accuracy score: 0.9394738118723824
F1 Score: 0.8376631825475993
250 Hz MultiBinary (Temporal):
Accuracy score: 0.9005061407976973
F1 Score: 0.5997235097991381
250 Hz MultiOutput (Temporal):
Accuracy score: 0.8922833741641805
F1 Score: 0.8674482897267625


In [13]:
from xgboost import XGBClassifier

features = np.load('../data/x_train_250_temporal_2s.npy')
labels = np.load('../data/y_train_binary_250_temporal_2s.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)

# Average CV score on the training set was: 0.8866608946608945
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.5, min_samples_leaf=2, min_samples_split=12, n_estimators=100)),
    XGBClassifier(learning_rate=0.5, max_depth=10, min_child_weight=17, n_estimators=100, n_jobs=1, subsample=1.0, verbosity=0)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250 Hz Binary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

features = np.load('../data/x_train_250_temporal_2s_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_250_temporal_2s_unrolled.npy')

# Average CV score on the training set was: 0.8563970405945847
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=MultinomialNB(alpha=0.01, fit_prior=True)),
    ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=1.0, min_samples_leaf=2, min_samples_split=16, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250 Hz MultiBinary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

/home/thoriri/miniconda3/envs/TPOT/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


250 Hz Binary (Temporal):
Accuracy score: 0.9363215937388829
F1 Score: 0.8385668607034517
250 Hz MultiBinary (Temporal):
Accuracy score: 0.9297888166618156
F1 Score: 0.8156576377685318


In [14]:
from sklearn.preprocessing import RobustScaler
features = np.load('../data/x_train_250_1000_temporal.npy')
labels = np.load('../data/y_train_binary_250_1000_temporal.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.9338566227989687
exported_pipeline = make_pipeline(
    RobustScaler(),
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.45, min_samples_leaf=3, min_samples_split=20, n_estimators=100)),
    ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.9500000000000001, min_samples_leaf=9, min_samples_split=14, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250Hz + 1000Hz Binary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))




features = np.load('../data/x_train_250_1000_temporal_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_250_1000_temporal_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.9008170970746223
exported_pipeline = ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.6000000000000001, min_samples_leaf=1, min_samples_split=15, n_estimators=100)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250Hz + 1000Hz MultiBinary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))


features = np.load('../data/x_train_250_1000_temporal_unrolled.npy')
labels = np.load('../data/y_train_multioutput_250_1000_temporal_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8915623043411454
exported_pipeline = ExtraTreesClassifier(bootstrap=True, criterion="gini", max_features=0.55, min_samples_leaf=1, min_samples_split=7, n_estimators=100)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)



exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("250Hz + 1000Hz MultiOutput (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))


250Hz + 1000Hz Binary (Temporal):
Accuracy score: 0.9343167851742222
F1 Score: 0.8247263361236317
250Hz + 1000Hz MultiBinary (Temporal):
Accuracy score: 0.901410829767382
F1 Score: 0.607776
250Hz + 1000Hz MultiOutput (Temporal):
Accuracy score: 0.8924745021073968
F1 Score: 0.8678263188915994


In [15]:
features = np.load('../data/x_train_256_temporal.npy')
labels = np.load('../data/y_train_binary_256_temporal.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)


# Average CV score on the training set was: 0.8729259174266039
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.6000000000000001, min_samples_leaf=2, min_samples_split=12, n_estimators=100)),
    GradientBoostingClassifier(learning_rate=0.1, max_depth=7, max_features=0.2, min_samples_leaf=14, min_samples_split=15, n_estimators=100, subsample=0.8500000000000001)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz Binary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

from xgboost import XGBClassifier
features = np.load('../data/x_train_256_temporal_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_256_temporal_unrolled.npy')
# Average CV score on the training set was: 0.880274417100841
exported_pipeline = XGBClassifier(learning_rate=0.1, max_depth=10, min_child_weight=7, n_estimators=100, n_jobs=1, subsample=0.9000000000000001, verbosity=0)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz MultiBinary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))

from sklearn.preprocessing import PolynomialFeatures
features = np.load('../data/x_train_256_temporal_unrolled.npy')
labels = np.load('../data/y_train_multioutput_256_temporal_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8688001974186669
exported_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    VarianceThreshold(threshold=0.1),
    ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.2, min_samples_leaf=1, min_samples_split=12, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)




exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz MultiOutput (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))

256Hz Binary (Temporal):
Accuracy score: 0.8734061440791951
F1 Score: 0.6440780517319619


/home/thoriri/miniconda3/envs/TPOT/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


256Hz MultiBinary (Temporal):
Accuracy score: 0.867541830311508
F1 Score: 0.6067092651757188
256Hz MultiOutput (Temporal):
Accuracy score: 0.8692231129283908
F1 Score: 0.8383738302151844


In [16]:
features = np.load('../data/x_train_256_512_temporal.npy')
labels = np.load('../data/y_train_binary_256_512_temporal.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)

# Average CV score on the training set was: 0.8668494119727651
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.7500000000000001, min_samples_leaf=4, min_samples_split=9, n_estimators=100)),
    StandardScaler(),
    ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.7500000000000001, min_samples_leaf=6, min_samples_split=9, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz + 512Hz Binary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))



features = np.load('../data/x_train_256_512_temporal_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_256_512_temporal_unrolled.npy')
# Average CV score on the training set was: 0.8712535534621267
exported_pipeline = ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.8500000000000001, min_samples_leaf=1, min_samples_split=14, n_estimators=100)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)
    
exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz + 512Hz MultiBinary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))



features = np.load('../data/x_train_256_512_temporal_unrolled.npy')
labels = np.load('../data/y_train_multioutput_256_512_temporal_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8551558332411835
exported_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.2, min_samples_leaf=1, min_samples_split=5, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("256Hz + 512Hz MultiOutput (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))

256Hz + 512Hz Binary (Temporal):
Accuracy score: 0.8688069758224336
F1 Score: 0.6709741550695824
256Hz + 512Hz MultiBinary (Temporal):
Accuracy score: 0.8651902497027348
F1 Score: 0.6467610022069324
256Hz + 512Hz MultiOutput (Temporal):
Accuracy score: 0.8568862794644073
F1 Score: 0.8258367676075989


In [17]:
from tpot.builtins import OneHotEncoder
features = np.load('../data/x_train_all_temporal.npy')
labels = np.load('../data/y_train_binary_all_temporal.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)

# Average CV score on the training set was: 0.9185302377742959
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.8, min_samples_leaf=3, min_samples_split=6, n_estimators=100)),
    DecisionTreeClassifier(criterion="entropy", max_depth=7, min_samples_leaf=14, min_samples_split=8)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("All frequencies Binary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))



features = np.load('../data/x_train_all_temporal_unrolled.npy')
labels = np.load('../data/y_train_multi_binary_all_temporal_unrolled.npy')
# Average CV score on the training set was: 0.8939939812444007
exported_pipeline = KNeighborsClassifier(n_neighbors=63, p=1, weights="distance")
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

    
exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("All frequencies MultiBinary (Temporal):")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results)))



features = np.load('../data/x_train_all_temporal_unrolled.npy')
labels = np.load('../data/y_train_multioutput_all_temporal_unrolled.npy')


training_features, testing_features, training_target, testing_target = \
            train_test_split(features, labels, random_state=42)
# Average CV score on the training set was: 0.8785453866190636
exported_pipeline = make_pipeline(
    OneHotEncoder(minimum_fraction=0.2, sparse=False, threshold=10),
    KNeighborsClassifier(n_neighbors=41, p=1, weights="distance")
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("All frequencies MultiOutput:")
print("Accuracy score: " + str(accuracy_score(testing_target,results)))
print("F1 Score: " + str(f1_score(testing_target,results, average='weighted')))


All frequencies Binary (Temporal):
Accuracy score: 0.9222641236641406
F1 Score: 0.7980088911151462
All frequencies MultiBinary (Temporal):
Accuracy score: 0.907239846263208
F1 Score: 0.7487681514113232
All frequencies MultiOutput:
Accuracy score: 0.8794108345005196
F1 Score: 0.8528066117658484
